# Download ERA5-LAND hourly temperatures and generate daily summary

# TODO: ERA5-Land for whole globe is something like 50GB per year: need different method to do daily summaries because 'naive' open+save causes Xarray to try to allocate RAM for the whole dataset. Might be possible to force it to avoid doing this e.g. map blocks, also need incremental write. Or just loop manually and save to netcdf instead of dicking about with xarray. Other option to do it on HPC cluster and just request really big RAM. 

# First Step is to download by Month instead of one big file. 

## Method

- Download hourly data for one whole year from ERA5 at 0.5deg resolution as GRIB file (in theory should be faster/smaller than netCDF and we will discard it anyway)
- Calculate minimum daily temperature for each day and save
- Delete the input GRIB to avoid acumulating terabytes of data!

In [1]:
from collections import namedtuple
from pathlib import Path

import numpy as np
import xarray as xr
import cdsapi

In [2]:
DATA_SRC = Path('~/Shared/Data/').expanduser()

In [3]:
SUBDAILY_TEMPERATURES_FOLDER = DATA_SRC / 'weather' / 'era5_land' / 'hourly_temperature_2m'
SUBDAILY_TEMPERATURES_FOLDER.mkdir(exist_ok=True)

TEMPERATURE_SUMMARY_FOLDER = DATA_SRC / 'weather' / 'era5_land' / 'era5_land_daily_summary'
TEMPERATURE_SUMMARY_FOLDER.mkdir(exist_ok=True)

In [4]:
MAX_YEAR = 2022

In [5]:
assert SUBDAILY_TEMPERATURES_FOLDER.is_dir()
assert TEMPERATURE_SUMMARY_FOLDER.is_dir()

In [6]:
def retreive_year(out_file, year):
    c = cdsapi.Client()
    c.retrieve(
        'reanalysis-era5-land',
        {
            'variable':'2m_temperature',
            'year': year,
            'month':[
                '01','02','03',
                '04','05','06',
                '07','08','09',
                '10','11','12'
            ],
            'day':[
                '01','02','03',
                '04','05','06',
                '07','08','09',
                '10','11','12',
                '13','14','15',
                '16','17','18',
                '19','20','21',
                '22','23','24',
                '25','26','27',
                '28','29','30',
                '31'
            ],
            'time':[
                '00:00','01:00','02:00',
                '03:00','04:00','05:00',
                '06:00','07:00','08:00',
                '09:00','10:00','11:00',
                '12:00','13:00','14:00',
                '15:00','16:00','17:00',
                '18:00','19:00','20:00',
                '21:00','22:00','23:00'
            ],
            'format':'grib'
        },
        str(out_file))
    
    
def generate_daily_summary(source_file):
    # resample and save a single year.
    # NOTE: CANNOT load at first because ERA5 LAND is 45GB per year!!
    daily = xr.open_dataset(source_file, engine='cfgrib')
    daily = daily.resample(time='1D')
    tmin = daily.min()
    tmax = daily.max()
    tmean = daily.mean()

    tmin = tmin.rename({'t2m': 't_min'})
    tmax = tmax.rename({'t2m': 't_max'})
    tmean = tmean.rename({'t2m': 't_mean'})
    daily_summary = xr.merge([tmin, tmax, tmean])
    return daily_summary


def download_and_summarise_year(year, overwrite=False):
    '''Download the GRIB file for hourly temperatures for one year
    Generate a summary file with temperature min, max, mean
    
    TODO: later, probably can delete the hourly data file after summarising it
    '''
    year = str(year)
    out_file = SUBDAILY_TEMPERATURES_FOLDER / f'{year}_temperature.grib'
    summary_file = TEMPERATURE_SUMMARY_FOLDER / f'{year}_temperature_summary.nc'
    
    if overwrite is True or out_file.exists() is False:
        retreive_year(out_file, year)
    else:
        print(f'Skip {out_file}, already exists.')
    
    if overwrite is True or summary_file.exists() is False:
        daily_summary = generate_daily_summary(out_file)
        
        # Encode using rounding to nearest 0.01K to save space, looses a small amount of precision
        # but should be ok since it not used for detailed simulations...
        daily_summary.to_netcdf(summary_file,
                            encoding={
                                't_min': {'dtype': 'int16', 'scale_factor': 0.01, '_FillValue': -9999},
                                't_max': {'dtype': 'int16', 'scale_factor': 0.01, '_FillValue': -9999},
                                't_mean': {'dtype': 'int16', 'scale_factor': 0.01, '_FillValue': -9999}
                            })
                           
    else:
        print(f'Skip {summary_file}, already exists.')

In [7]:
# EXAMPLE code for triggering many requests.
# joblib.Parallel(n_jobs=10, verbose=10)(
#     joblib.delayed(download_and_summarise_year)(year) for year in range(1980, 2020))

# Run each year one at a time

Because this is anyway a long process, only do one year at a time - just set it going and leave it!

Also, for the summarizing step the whole dataset is loaded into memory because it is anyway needed to get min/max/mean. So can't really do several in parallel b/c it will blow up the RAM!

In [8]:
# for year in range(1980, MAX_YEAR+1):
#     print(year)
#     download_and_summarise_year(year)

In [9]:
def retreive_month(out_file, year, month):
    if out_file.exists():
        print('file exists, skipping')
        return
    
    c = cdsapi.Client()
    c.retrieve(
        'reanalysis-era5-land',
        {
            'variable':'2m_temperature',
            'year': year,
            'month':[str(month).zfill(2)],
            'day':[
                '01','02','03',
                '04','05','06',
                '07','08','09',
                '10','11','12',
                '13','14','15',
                '16','17','18',
                '19','20','21',
                '22','23','24',
                '25','26','27',
                '28','29','30',
                '31'
            ],
            'time':[
                '00:00','01:00','02:00',
                '03:00','04:00','05:00',
                '06:00','07:00','08:00',
                '09:00','10:00','11:00',
                '12:00','13:00','14:00',
                '15:00','16:00','17:00',
                '18:00','19:00','20:00',
                '21:00','22:00','23:00'
            ],
            'format':'grib'
        },
        str(out_file))
    


In [10]:
year = MAX_YEAR

In [ ]:
for month in range(1,13):
    out_file = SUBDAILY_TEMPERATURES_FOLDER / f'{year}-{str(month).zfill(2)}_temperature_2m.grib'
    retreive_month(out_file, year, month)

2023-05-31 08:36:45,150 INFO Welcome to the CDS
2023-05-31 08:36:45,152 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-land
2023-05-31 08:36:45,336 INFO Request is queued
2023-05-31 08:36:46,361 INFO Request is running
2023-05-31 08:38:00,446 INFO Request is completed
2023-05-31 08:38:00,447 INFO Downloading https://download-0000-clone.copernicus-climate.eu/cache-compute-0000/cache/data5/adaptor.mars.internal-1685515005.7654817-7784-12-178a0816-a7f1-4a4a-806d-aec5d7d030a0.grib to /home/jonathanchambers/Shared/Data/weather/era5_land/hourly_temperature_2m/2022-01_temperature_2m.grib (3.6G)
2023-05-31 08:45:31,738 INFO Download rate 8.2M/s   
2023-05-31 08:45:31,898 INFO Welcome to the CDS
2023-05-31 08:45:31,899 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-land
2023-05-31 08:45:31,970 INFO Request is queued
2023-05-31 08:45:32,991 INFO Request is running
2023-05-31 08:46:47,045 INFO Request is compl

In [16]:
# var = 'max'
# daily_folder = TEMPERATURE_SUMMARY_FOLDER / f'{year}' / var
# daily_folder.mkdir(parents=True, exist_ok=True)

In [ ]:
# summary_file = daily_folder / f'era5-land_global_daily_t{var}_{year}{str(month).zfill(2)}.nc'

# hourly_temperatures = xr.open_dataset(out_file, chunks=dict(step=-1, longitude=10))

# ref = xr.open_dataset('/home/jonathanchambers/Shared/Data/weather/era5_land/era5_land_daily_summary/1979/max/era5-land_global_daily_tmax_197901.nc')

# encoding = ref.tmax.encoding.copy()
# encoding.pop('source')

# daily_max = hourly_temperatures.max(dim='step').drop('surface').drop('number')
# daily_max = daily_max.rename({'t2m': f't{var}'})
# daily_max.attrs['frequency'] = 'day'

# daily_max.to_netcdf(summary_file, 
#                     encoding={f't{var}':encoding})

  
  
```shell
grib_to_netcdf -k 3 -d 1 2021-01_temperature_2m.grib -o 2021-01_temperature_2m.nc
  
cdo -daymax -chname,t2m,tmax 2021-01_temperature_2m.nc era5-land_global_daily_tmax_19790101.nc
  
```

In [ ]:
base_path / 'hourly_nc'

In [17]:
import subprocess
from tqdm.notebook import tqdm

In [30]:
TEMPERATURE_SUMMARY_FOLDER

PosixPath('/home/jonathanchambers/Shared/Data/weather/era5_land/era5_land_daily_summary')

In [37]:
year = MAX_YEAR
base_path = Path('~/Scratch').expanduser() / 'proc_era5_land_tmp'
# daily_folder = base_path / 'daily'
daily_folder = TEMPERATURE_SUMMARY_FOLDER


grib_to_netcdf = '/home/jonathanchambers/Scratch/.conda/envs/science2/bin/grib_to_netcdf'

for month in tqdm(list(range(1, 13))):
    month = str(month).zfill(2)
    
    # input_file = base_path / f'{year}-{month}_temperature_2m.grib'
    # hourly_nc_file = base_path / 'hourly_nc' / f'{year}-{month}_temperature_2m.nc'
    input_file = SUBDAILY_TEMPERATURES_FOLDER / f'{year}-{month}_temperature_2m.grib'
    hourly_nc_file = DATA_SRC / 'weather' / 'era5_land' / 'hourly_nc' / f'{year}-{month}_temperature_2m.nc'

    to_netcdf = [grib_to_netcdf, '-k', '3', '-d', '3', str(input_file), '-o', str(hourly_nc_file) ]

    if not hourly_nc_file.exists():
        subprocess.run(to_netcdf, shell=False, check=True)
        
    for var in tqdm(['mean', 'min', 'max']):
        summary_file = daily_folder / f'{year}' / f'{var}' / f'era5-land_global_daily_t{var}_{year}{str(month).zfill(2)}.nc'
        
        daily_summary = ['cdo', f'-day{var}', f'-chname,t2m,t{var}', str(hourly_nc_file), str(summary_file)]

        if not summary_file.exists():
            subprocess.run(daily_summary, shell=False, check=True)

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

cdo(1) chname: Process started
cdo(1) chname: Processed 4823798400 values from 1 variable over 744 timesteps.
cdo    daymean: Processed 4823798400 values from 1 variable over 774 timesteps [72.10s 412MB].
cdo(1) chname: Process started
cdo(1) chname: Processed 4823798400 values from 1 variable over 744 timesteps.
cdo    daymin: Processed 4823798400 values from 1 variable over 774 timesteps [70.67s 422MB].
grib_to_netcdf: Version 2.28.0
grib_to_netcdf: Processing input file '/home/jonathanchambers/Shared/Data/weather/era5_land/hourly_temperature_2m/2022-02_temperature_2m.grib'.
grib_to_netcdf: Found 672 GRIB fields in 1 file.
grib_to_netcdf: Ignoring key(s): method, type, stream, refdate, hdate
grib_to_netcdf: Creating netCDF file '/home/jonathanchambers/Shared/Data/weather/era5_land/hourly_nc/2022-02_temperature_2m.nc'
grib_to_netcdf: NetCDF library version: 4.8.1 of Oct 31 2022 22:16:44 $
grib_to_netcdf: Creating netCDF-4/HDF5 format.
grib_to_netcdf: Defining variable 't2m'.
grib_to_n

  0%|          | 0/3 [00:00<?, ?it/s]

cdo(1) chname: Process started
cdo(1) chname: Processed 4356979200 values from 1 variable over 672 timesteps.
cdo    daymean: Processed 4356979200 values from 1 variable over 699 timesteps [66.06s 413MB].
cdo(1) chname: Process started
cdo(1) chname: Processed 4356979200 values from 1 variable over 672 timesteps.
cdo    daymin: Processed 4356979200 values from 1 variable over 699 timesteps [65.70s 413MB].
cdo(1) chname: Process started
cdo(1) chname: Processed 4356979200 values from 1 variable over 672 timesteps.
cdo    daymax: Processed 4356979200 values from 1 variable over 699 timesteps [64.65s 413MB].
grib_to_netcdf: Version 2.28.0
grib_to_netcdf: Processing input file '/home/jonathanchambers/Shared/Data/weather/era5_land/hourly_temperature_2m/2022-03_temperature_2m.grib'.
grib_to_netcdf: Found 744 GRIB fields in 1 file.
grib_to_netcdf: Ignoring key(s): method, type, stream, refdate, hdate
grib_to_netcdf: Creating netCDF file '/home/jonathanchambers/Shared/Data/weather/era5_land/ho

...

cdo(1) chname: Process started
cdo(1) chname: Processed 4823798400 values from 1 variable over 744 timesteps.
cdo    daymean: Processed 4823798400 values from 1 variable over 774 timesteps [73.87s 420MB].
cdo(1) chname: Process started
cdo(1) chname: Processed 4823798400 values from 1 variable over 744 timesteps.
cdo    daymin: Processed 4823798400 values from 1 variable over 774 timesteps [75.89s 444MB].
cdo(1) chname: Process started
cdo(1) chname: Processed 4823798400 values from 1 variable over 744 timesteps.
cdo    daymax: Processed 4823798400 values from 1 variable over 774 timesteps [73.70s 445MB].
grib_to_netcdf: Version 2.28.0
grib_to_netcdf: Processing input file '/home/jonathanchambers/Shared/Data/weather/era5_land/hourly_temperature_2m/2022-11_temperature_2m.grib'.
grib_to_netcdf: Found 720 GRIB fields in 1 file.
grib_to_netcdf: Ignoring key(s): method, type, stream, refdate, hdate
grib_to_netcdf: Creating netCDF file '/home/jonathanchambers/Shared/Data/weather/era5_land/ho

  0%|          | 0/3 [00:00<?, ?it/s]

cdo(1) chname: Process started
cdo(1) chname: Processed 4668192000 values from 1 variable over 720 timesteps.
cdo    daymean: Processed 4668192000 values from 1 variable over 749 timesteps [66.96s 448MB].
cdo(1) chname: Process started
cdo(1) chname: Processed 4668192000 values from 1 variable over 720 timesteps.
cdo    daymin: Processed 4668192000 values from 1 variable over 749 timesteps [69.95s 435MB].
cdo(1) chname: Process started
cdo(1) chname: Processed 4668192000 values from 1 variable over 720 timesteps.
cdo    daymax: Processed 4668192000 values from 1 variable over 749 timesteps [63.52s 435MB].
grib_to_netcdf: Version 2.28.0
grib_to_netcdf: Processing input file '/home/jonathanchambers/Shared/Data/weather/era5_land/hourly_temperature_2m/2022-12_temperature_2m.grib'.
grib_to_netcdf: Found 744 GRIB fields in 1 file.
grib_to_netcdf: Ignoring key(s): method, type, stream, refdate, hdate
grib_to_netcdf: Creating netCDF file '/home/jonathanchambers/Shared/Data/weather/era5_land/ho

  0%|          | 0/3 [00:00<?, ?it/s]

cdo(1) chname: Process started
cdo(1) chname: Processed 4823798400 values from 1 variable over 744 timesteps.
cdo    daymean: Processed 4823798400 values from 1 variable over 774 timesteps [72.06s 413MB].
cdo(1) chname: Process started
cdo(1) chname: Processed 4823798400 values from 1 variable over 744 timesteps.
cdo    daymin: Processed 4823798400 values from 1 variable over 774 timesteps [73.92s 426MB].
cdo(1) chname: Process started
cdo(1) chname: Processed 4823798400 values from 1 variable over 744 timesteps.
cdo    daymax: Processed 4823798400 values from 1 variable over 774 timesteps [71.69s 426MB].


In [38]:
year = MAX_YEAR
base_path = Path('~/Scratch').expanduser() / 'proc_era5_land_tmp'
# daily_folder = base_path / 'daily'
daily_folder = TEMPERATURE_SUMMARY_FOLDER


grib_to_netcdf = '/home/jonathanchambers/Scratch/.conda/envs/science2/bin/grib_to_netcdf'

for month in tqdm(list(range(1, 2))):
    month = str(month).zfill(2)
    
    # input_file = base_path / f'{year}-{month}_temperature_2m.grib'
    # hourly_nc_file = base_path / 'hourly_nc' / f'{year}-{month}_temperature_2m.nc'
    input_file = SUBDAILY_TEMPERATURES_FOLDER / f'{year}-{month}_temperature_2m.grib'
    hourly_nc_file = DATA_SRC / 'weather' / 'era5_land' / 'hourly_nc' / f'{year}-{month}_temperature_2m.nc'

    to_netcdf = [grib_to_netcdf, '-k', '3', '-d', '3', str(input_file), '-o', str(hourly_nc_file) ]

    if not hourly_nc_file.exists():
        subprocess.run(to_netcdf, shell=False, check=True)
        
    for var in tqdm(['max']):
        summary_file = daily_folder / f'{year}' / f'{var}' / f'era5-land_global_daily_t{var}_{year}{str(month).zfill(2)}.nc'
        
        daily_summary = ['cdo', f'-day{var}', f'-chname,t2m,t{var}', str(hourly_nc_file), str(summary_file)]

        # if not summary_file.exists():
        subprocess.run(daily_summary, shell=False, check=True)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

cdo(1) chname: Process started
cdo(1) chname: Processed 4823798400 values from 1 variable over 744 timesteps.
cdo    daymax: Processed 4823798400 values from 1 variable over 774 timesteps [85.59s 422MB].


In [29]:
encoding

NameError: name 'encoding' is not defined

In [52]:
new_encoding = {'zlib': True,
 'shuffle': True,
 'complevel': 1,
 'chunksizes': (1, 1801, 3600),
 'original_shape': (31, 1801, 3600),
 'dtype': np.dtype('int32'),
 'missing_value': -9999,
 '_FillValue': -9999,
 'scale_factor': 0.0016127533931795336,
 'add_offset': 264.7066521193972}

In [57]:
out_folder = Path('/home/jonathanchambers/Shared/Data/weather/era5_land/era5_land_daily_summary/2022/')
for month in tqdm(list(range(1, 13))):
    month = str(month).zfill(2)
    for var in tqdm(['mean', 'min', 'max']):
        summary_file = daily_folder / f't{var}' / f'era5-land_global_daily_t{var}_{year}{str(month).zfill(2)}.nc'
        re_save_file = out_folder / var /f'era5-land_global_daily_t{var}_{year}{str(month).zfill(2)}.nc'
        data = xr.open_dataset(summary_file)
        data.to_netcdf(re_save_file, encoding={f't{var}': new_encoding})

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

In [42]:
# subprocess.run(task_list['daymax'], shell=False, check=True)

In [35]:
# subprocess.run(task_list['to_netcdf'], shell=False, check=True)

grib_to_netcdf: Version 2.28.0
grib_to_netcdf: Processing input file '/home/jonathanchambers/Scratch/proc_era5_land_tmp/2021-01_temperature_2m.grib'.
grib_to_netcdf: Found 744 GRIB fields in 1 file.
grib_to_netcdf: Ignoring key(s): method, type, stream, refdate, hdate
grib_to_netcdf: Creating netCDF file '/home/jonathanchambers/Scratch/proc_era5_land_tmp/hourly_nc/2021-01_temperature_2m.nc'
grib_to_netcdf: NetCDF library version: 4.8.1 of Oct 31 2022 22:16:44 $
grib_to_netcdf: Creating netCDF-4/HDF5 format.
grib_to_netcdf: Defining variable 't2m'.
grib_to_netcdf: Done.


CompletedProcess(args=['/home/jonathanchambers/Scratch/.conda/envs/science2/bin/grib_to_netcdf', '-k', '3', '-d', '3', '/home/jonathanchambers/Scratch/proc_era5_land_tmp/2021-01_temperature_2m.grib', '-o', '/home/jonathanchambers/Scratch/proc_era5_land_tmp/hourly_nc/2021-01_temperature_2m.nc'], returncode=0)